In [1]:
# Comprobar rutas y modelos 
#import os
from pathlib import Path

# Define las rutas de las carpetas
MODEL_DIR = Path("models")
DATASET_DIR = Path("dataset")
OUTPUT_DIR = Path("output")

# Define las rutas de los tres archivos de modelo
FACE_PROTO = MODEL_DIR / "deploy.prototxt"
FACE_MODEL = MODEL_DIR / "res10_300x300_ssd_iter_140000.caffemodel"
EMBED_MODEL = MODEL_DIR / "openface_nn4.small2.v1.t7"

def check_paths():
    ok = True
    print("Verificando archivos y carpetas...\n")

    # Verificar los 3 modelos
    for p in [FACE_PROTO, FACE_MODEL, EMBED_MODEL]:
        if not p.exists():
            print(f"[FALTA] {p}")
            ok = False
        else:
            print(f"[OK] {p}")

    # Verificar dataset y output
    if not DATASET_DIR.exists():
        print(f"[FALTA] dataset/ (creando) → {DATASET_DIR.resolve()}")
        DATASET_DIR.mkdir(parents=True, exist_ok=True)
    else:
        print(f"[OK] dataset/: {DATASET_DIR.resolve()}")

    if not OUTPUT_DIR.exists():
        print(f"[FALTA] output/ (creando) → {OUTPUT_DIR.resolve()}")
        OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    else:
        print(f"[OK] output/: {OUTPUT_DIR.resolve()}")

    return ok

# Ejecuta la verificación
_ = check_paths()

if check_paths() == True:
    print("\n Todos los modelos están en [OK], continuar.")
else:
    print("\n Todos los modelos están en [OK], continuar.")
    print(" Si alguno dice [FALTA], revisa el nombre o su ubicación en la carpeta 'models/'.")


Verificando archivos y carpetas...

[OK] models\deploy.prototxt
[OK] models\res10_300x300_ssd_iter_140000.caffemodel
[OK] models\openface_nn4.small2.v1.t7
[OK] dataset/: C:\Users\Santi\Documents\FaceReco\dataset
[FALTA] output/ (creando) → C:\Users\Santi\Documents\FaceReco\output
Verificando archivos y carpetas...

[OK] models\deploy.prototxt
[OK] models\res10_300x300_ssd_iter_140000.caffemodel
[OK] models\openface_nn4.small2.v1.t7
[OK] dataset/: C:\Users\Santi\Documents\FaceReco\dataset
[OK] output/: C:\Users\Santi\Documents\FaceReco\output

 Todos los modelos están en [OK], continuar.


In [2]:
# Cargar modelos y definir funciones de detección y embeddings ---
import cv2
import numpy as np

# Cargar el detector de caras (SSD con ResNet-10)
detector = cv2.dnn.readNetFromCaffe(str(FACE_PROTO), str(FACE_MODEL))

# Cargar el modelo de embeddings (OpenFace)
embedder = cv2.dnn.readNetFromTorch(str(EMBED_MODEL))

print(" Todos los modelos cargados correctamente.")


# Función para detectar la cara más grande en una imagen
def detectar_cara_bgr(image_bgr, conf_thresh=0.5, min_side=40):
    (h, w) = image_bgr.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(image_bgr, (300, 300)),
                                 1.0, (300, 300),
                                 (104.0, 177.0, 123.0))
    detector.setInput(blob)
    detections = detector.forward()
    best = None
    best_area = 0
    for i in range(detections.shape[2]):
        conf = detections[0, 0, i, 2]
        if conf < conf_thresh:
            continue

        x1, y1, x2, y2 = (detections[0, 0, i, 3:7] * np.array([w, h, w, h])).astype(int)

        # Recorte seguro a límites
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(w - 1, x2), min(h - 1, y2)
        
        area = (x2 - x1) * (y2 - y1)
        if area > best_area:
            best_area = area
            best = (x1, y1, x2, y2, conf)
    return best  # None si no hay rostro válido


# Función para obtener el embedding (vector 128-D)
def embedding_cara(image_bgr, box, margin=0.2):
    if box is None:
        return None
    x1, y1, x2, y2, _ = box
    h, w = image_bgr.shape[:2]

    # Margen alrededor del rostro
    bw, bh = x2 - x1, y2 - y1
    dx, dy = int(bw * margin), int(bh * margin)
    x1m, y1m = max(0, x1 - dx), max(0, y1 - dy)
    x2m, y2m = min(w, x2 + dx), min(h, y2 + dy)

    if x2m <= x1m or y2m <= y1m:
        return None
        
    face = image_bgr[y1m:y2m, x1m:x2m]
    if face is None or face.size == 0:
        return None

    face_blob = cv2.dnn.blobFromImage(cv2.resize(face, (96, 96)),
                                      1.0/255, (96, 96),
                                      (0, 0, 0), swapRB=True, crop=False)
    embedder.setInput(face_blob)
    vec = embedder.forward()
    if vec is None or vec.size == 0:
        return None
    return vec.flatten()

 Todos los modelos cargados correctamente.


In [3]:
# Extraer frames de varios videos, eligiendo carpeta y número de imágenes
from tkinter import Tk, filedialog
import cv2
from pathlib import Path
import re

# Configurar Tkinter para selección de archivos
root = Tk()
root.withdraw()

def slugify(text: str) -> str:
    """Convierte a nombre de carpeta seguro (sin espacios ni caracteres raros)."""
    text = text.strip().replace(" ", "_")
    return re.sub(r"[^A-Za-z0-9_\-]", "", text) or "frames_extraidos"

def extraer_frames_de_video(video_path: str, destino_dir: Path, n_frames: int) -> int:
    """Extrae n_frames equiespaciados del video y los guarda en destino_dir."""
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ No se pudo abrir el video: {video_path}")
        return 0

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) or 0
    fps = cap.get(cv2.CAP_PROP_FPS) or 0
    dur = (total_frames / fps) if (fps > 0 and total_frames > 0) else 0
    print(f"🎞 {Path(video_path).name} → {total_frames} frames | {fps:.2f} fps | {dur:.2f} s")

    if total_frames <= 0:
        cap.release()
        print("⚠️ El video no reporta frames legibles.")
        return 0

    n = min(n_frames, total_frames)
    indices = [int(round(i * (total_frames - 1) / (n - 1))) for i in range(n)] if n > 1 else [0]

    destino_dir.mkdir(parents=True, exist_ok=True)

    saved = 0
    for idx in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ok, frame = cap.read()
        if not ok:
            print(f"[AVISO] No se pudo leer el frame {idx}")
            continue
        out_file = destino_dir / f"frame_{idx:06d}.jpg"
        cv2.imwrite(str(out_file), frame)
        saved += 1

    cap.release()
    return saved

# Bucle principal
while True:
    # Seleccionar video
    video_path = filedialog.askopenfilename(
        title="Selecciona un video",
        filetypes=[("Videos", "*.mp4 *.avi *.mov *.mkv"), ("Todos los archivos", "*.*")]
    )
    if not video_path:
        print("🚪 No seleccionaste video. Fin del proceso.")
        break

    # Pedir número de imágenes
    try:
        n_frames = int(input("¿Cuántas imágenes deseas extraer? (Recomendable = 200): ").strip() or "200")
    except ValueError:
        n_frames = 200

    # Pedir nombre de carpeta
    nombre_carpeta = input("👉 Escribe el nombre de la carpeta donde guardar las imágenes: ").strip()
    nombre_carpeta = slugify(nombre_carpeta)
    destino_dir = Path.cwd() / "dataset" / nombre_carpeta
    print(f"📁 Guardando imágenes en: {destino_dir.resolve()}")

    # Extraer frames
    guardadas = extraer_frames_de_video(video_path, destino_dir, n_frames)
    print(f"✅ Se guardaron {guardadas} imágenes en '{destino_dir}'")

    # Preguntar si continuarV
    seguir = input("¿Quieres procesar otro video? (s/n): ").strip().lower()
    if seguir not in ("s", "si", "sí"):
        print("✔️ Proceso finalizado.")
        break

¿Cuántas imágenes deseas extraer? (Recomendable = 200):  200
👉 Escribe el nombre de la carpeta donde guardar las imágenes:  Santiago_Zagal


📁 Guardando imágenes en: C:\Users\Santi\Documents\FaceReco\dataset\Santiago_Zagal
🎞 WhatsApp Video 2025-11-08 at 08.13.52.mp4 → 463 frames | 30.00 fps | 15.43 s
✅ Se guardaron 200 imágenes en 'C:\Users\Santi\Documents\FaceReco\dataset\Santiago_Zagal'


¿Quieres procesar otro video? (s/n):  S


🚪 No seleccionaste video. Fin del proceso.


In [4]:
# Generar embeddings de todas las imágenes en dataset/ (reemplazando archivo si ya existe) 
import os
import cv2
import csv
import sys
import pickle
import shutil
from collections import Counter
from imutils import paths
from pathlib import Path
from datetime import datetime

# CONFIGURACIÓN
#DATASET_DIR = Path("dataset")
BASE_DIR = Path.cwd()
OUTPUT_DIR = Path("output")
EMB_PATH    = OUTPUT_DIR / "embeddings.pickle"
#LOG_DIR     = DATASET_DIR / "diagnostico"
LOG_DIR     = BASE_DIR / "diagnostico"
NOFACE_DIR  = LOG_DIR / "sin_rostro"
EMBFAIL_DIR = LOG_DIR / "embedding_fallido"
CSV_FALLOS  = LOG_DIR / "fallos.csv"

# Rotaciones y umbrales
CONF_LIST  = [0.50, 0.35, 0.25]
ROTATIONS  = [0, 90, 180, 270]  # grados
MIN_SIDE   = 40                 # tamaño mínimo de cara (px)
MARGIN     = 0.20               # margen alrededor del bbox para el embedding

# Asegurar carpetas
if not DATASET_DIR.exists():
    raise FileNotFoundError(f"No existe la carpeta del dataset: {DATASET_DIR}")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
LOG_DIR.mkdir(parents=True, exist_ok=True)
NOFACE_DIR.mkdir(parents=True, exist_ok=True)
EMBFAIL_DIR.mkdir(parents=True, exist_ok=True)

def rotar_bgr(img_bgr, angle):
    if angle == 0:
        return img_bgr
    elif angle == 90:
        return cv2.rotate(img_bgr, cv2.ROTATE_90_CLOCKWISE)
    elif angle == 180:
        return cv2.rotate(img_bgr, cv2.ROTATE_180)
    else:  # 270
        return cv2.rotate(img_bgr, cv2.ROTATE_90_COUNTERCLOCKWISE)

def safe_print(*args, **kwargs):
    """Print con flush forzado para evitar cortes/renglones truncados."""
    print(*args, **kwargs)
    sys.stdout.flush()

# Listar imágenes (excluir checkpoints/ocultas)
image_paths = [
    p for p in paths.list_images(str(DATASET_DIR))
    if ".ipynb_checkpoints" not in p and not Path(p).parent.name.startswith(".")
]
if not image_paths:
    raise RuntimeError(f"⚠️ No se encontraron imágenes dentro de {DATASET_DIR}")

safe_print(f"🔍 Procesando {len(image_paths)} imágenes...\n")

embeddings, labels = [], []
fail_reasons = Counter()
fallos_rows = []  # para CSV

# Procesar
for (i, img_path) in enumerate(image_paths, start=1):
    img_path = Path(img_path)
    person = img_path.parent.name
    img0 = cv2.imread(str(img_path))

    if img0 is None:
        safe_print(f"[❌] No se pudo leer {img_path}")
        fail_reasons["imread"] += 1
        fallos_rows.append([str(img_path), "imread", "No se pudo leer"])
        # Mover a diagnostico
        try:
            shutil.copy2(str(img_path), LOG_DIR / img_path.name)
        except Exception:
            pass
        continue

    success = False
    bbox_usada = None

    for angle in ROTATIONS:
        img = rotar_bgr(img0, angle)

        # Intentar detección con distintos umbrales
        box = None
        for thr in CONF_LIST:
            try:
                box = detectar_cara_bgr(img, conf_thresh=thr, min_side=MIN_SIDE)
            except Exception as e:
                box = None
            if box is not None:
                break

        if box is None:
            continue  # probar otra rotación

        # Intentar embedding con margen
        try:
            vec = embedding_cara(img, box, margin=MARGIN)
        except Exception as e:
            vec = None

        if vec is not None:
            embeddings.append(vec)
            labels.append(person)
            success = True
            bbox_usada = box
            break  # ¡listo para esta imagen!

    if not success:
        # Diagnóstico: ¿falló detección o embedding?
        box_dbg = None
        for thr in CONF_LIST:
            try:
                box_dbg = detectar_cara_bgr(img0, conf_thresh=thr, min_side=MIN_SIDE)
            except Exception:
                box_dbg = None
            if box_dbg is not None:
                break

        if box_dbg is None:
            safe_print(f"[⚠️] No se detectó rostro en {img_path}")
            fail_reasons["sin_rostro"] += 1
            fallos_rows.append([str(img_path), "sin_rostro", "Detección fallida (todas las rotaciones)"])
            # Copiar al folder de no-face (para inspección)
            try:
                shutil.copy2(str(img_path), NOFACE_DIR / img_path.name)
            except Exception:
                pass
        else:
            safe_print(f"[⚠️] No se pudo generar embedding para {img_path}")
            fail_reasons["embedding_fallido"] += 1
            fallos_rows.append([str(img_path), "embedding_fallido", "Encoder devolvió None"])
            # Copiar al folder de embedding-fallo
            try:
                shutil.copy2(str(img_path), EMBFAIL_DIR / img_path.name)
            except Exception:
                pass

    # Progreso cada 100 imágenes
    if (i % 100 == 0) or (i == len(image_paths)):
        safe_print(f"   → Procesadas {i}/{len(image_paths)} imágenes...")

# ===== Resultados =====
safe_print("\n RESULTADOS ")
safe_print(f"✅ Embeddings generados: {len(embeddings)}")
safe_print(f"👥 Personas detectadas: {sorted(set(labels))}")
safe_print(f"📊 Resumen de fallos: {dict(fail_reasons)}")

# Guardar CSV de fallos
if fallos_rows:
    with open(CSV_FALLOS, "w", newline="", encoding="utf-8") as fcsv:
        writer = csv.writer(fcsv)
        writer.writerow(["ruta_imagen", "tipo_fallo", "detalle"])
        writer.writerows(fallos_rows)
    safe_print(f"🧾 CSV de fallos: {CSV_FALLOS}")

# Guardar embeddings (reemplaza si existe)
if EMB_PATH.exists():
    try:
        os.remove(EMB_PATH)
        safe_print(f"⚙️ Archivo existente eliminado: {EMB_PATH.name}")
    except Exception as e:
        raise RuntimeError(f"No se pudo eliminar {EMB_PATH}: {e}")

payload = {"embeddings": embeddings, "names": labels, "fecha": datetime.utcnow().isoformat() + "Z"}
with open(EMB_PATH, "wb") as f:
    pickle.dump(payload, f, protocol=pickle.HIGHEST_PROTOCOL)

safe_print(f" 💾 Embeddings guardados en: {EMB_PATH.resolve()}")


🔍 Procesando 737 imágenes...

   → Procesadas 100/737 imágenes...
   → Procesadas 200/737 imágenes...
   → Procesadas 300/737 imágenes...
   → Procesadas 400/737 imágenes...
   → Procesadas 500/737 imágenes...
   → Procesadas 600/737 imágenes...
   → Procesadas 700/737 imágenes...
   → Procesadas 737/737 imágenes...

 RESULTADOS 
✅ Embeddings generados: 737
👥 Personas detectadas: ['Santiago_Zagal', 'Vinicio_Quilumba']
📊 Resumen de fallos: {}
 💾 Embeddings guardados en: C:\Users\Santi\Documents\FaceReco\output\embeddings.pickle


In [5]:
# Entrenar SVM y guardar artefactos (eliminar y reemplazar si existen)
import os, pickle, numpy as np, warnings

from sklearn.metrics.pairwise import cosine_distances
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder, Normalizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.exceptions import UndefinedMetricWarning

# Ignorar advertencias de métricas indefinidas 🚫
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

# Rutas
OUTPUT_DIR = Path("output")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
EMB_PATH = OUTPUT_DIR / "embeddings.pickle"
RECOGNIZER_PATH = OUTPUT_DIR / "recognizer_svm.pickle"
ENCODER_PATH = OUTPUT_DIR / "label_encoder.pickle"
THRESH_PATH = OUTPUT_DIR / "threshold.pkl"
OPEN_META = OUTPUT_DIR / "open_set_meta.pkl"

# 1) Cargar embeddings
with open(EMB_PATH, "rb") as f:
    data = pickle.load(f)

X = np.array(data["embeddings"])   # (n_muestras, 128)
names = np.array(data["names"])    # etiquetas de texto

print("Shape de X:", X.shape)
print("Clases:", sorted(set(names)))

# 2) Codificar etiquetas
le = LabelEncoder().fit(names)
y = le.transform(names)

# 3) Train/Test split si hay datos suficientes (>=2 clases y >=5 muestras)
n_clases = len(np.unique(y))
min_por_clase = min(Counter(y).values())
hacer_split = (n_clases >= 2) and (len(y) >= 5)

if hacer_split and min_por_clase >= 2:
    Xtr, Xte, ytr, yte = train_test_split(
        X, y, test_size=0.20, stratify=y, random_state=42
    )
else:
    Xtr, Xte, ytr, yte = X, X, y, y
    print("\n(Conjunto pequeño o con clases de 1 muestra: se entrenará sin split de validación)")

# 4) Pipeline: Normalización L2 + SVM y búsqueda de hiperparámetros
pipe = make_pipeline(
    Normalizer(norm="l2"),
    SVC(probability=True, class_weight="balanced", random_state=42)
)

param_grid = {
    "svc__kernel": ["linear", "rbf"],
    "svc__C": [0.1, 1, 10, 100],
    "svc__gamma": ["scale", 0.01, 0.001]  # gamma se usa solo con RBF
}

cv = StratifiedKFold(n_splits=min(5, max(2, min(Counter(ytr).values()))),
                     shuffle=True, random_state=42)

gs = GridSearchCV(pipe, param_grid, cv=cv, n_jobs=-1, verbose=0)
gs.fit(Xtr, ytr)
clf = gs.best_estimator_
print("➡️ Mejor configuración:", gs.best_params_)


# Normalizamos X con el mismo normalizer del pipeline
X_norm = clf.named_steps["normalizer"].transform(X)

class_means = {}
class_thr = {}

for idx_cls, cls_name in enumerate(le.classes_):
    idx = np.where(y == idx_cls)[0]
    Xc = X_norm[idx]
    mu = Xc.mean(axis=0)                 # centroide de la clase
    dists = cosine_distances(Xc, [mu]).ravel()
    
    # Umbral por clase: p.ej. percentil 95 de las distancias intra-clase
    t = np.quantile(dists, 0.95)

    class_means[cls_name] = mu
    class_thr[cls_name] = float(t)

open_set_meta = {
    "class_means": class_means,
    "class_thr": class_thr,
}

with open(OUTPUT_DIR / "open_set_meta.pkl", "wb") as f:
    pickle.dump(open_set_meta, f)


# 5) Evaluación y cálculo de umbral de confianza (para open-set)
def find_best_threshold(model, Xv, yv):
    """Devuelve el mejor umbral de probabilidad para etiquetar 'Desconocido'."""
    if len(Xv) == 0:
        return 0.7  # valor razonable por defecto
    probs = model.predict_proba(Xv)
    pmax = probs.max(axis=1)
    pred_idx = probs.argmax(axis=1)
    # probar umbrales y maximizar F1 macro en los casos aceptados
    best_t, best_f1 = 0.6, -1
    for t in np.linspace(0.5, 0.95, 10):
        aceptadas = pmax >= t
        if not np.any(aceptadas):
            continue
        f1 = f1_score(yv[aceptadas], pred_idx[aceptadas], average="macro", zero_division=0)
        if f1 > best_f1:
            best_f1, best_t = f1, t
    return float(best_t)

if hacer_split and (Xte is not Xtr):
    # Informe clásico
    ypred = clf.predict(Xte)
    print("\n=== Reporte de clasificación ===")
    print(classification_report(yte, ypred, target_names=le.classes_))
    print("Matriz de confusión:\n", confusion_matrix(yte, ypred))

    # Umbral aprendido en validación
    best_t = find_best_threshold(clf, Xte, yte)
else:
    best_t = 0.7  # por defecto si no hay validación

print(f"🔧 Umbral de confianza recomendado: {best_t:.3f}")

# 6) Eliminar existentes y guardar artefactos nuevos
for path in (RECOGNIZER_PATH, ENCODER_PATH, THRESH_PATH):
    if path.exists():
        os.remove(path)
        print(f"⚙️ Eliminado archivo existente: {path.name}")

with open(RECOGNIZER_PATH, "wb") as f:
    pickle.dump(clf, f)
with open(ENCODER_PATH, "wb") as f:
    pickle.dump(le, f)
with open(THRESH_PATH, "wb") as f:
    pickle.dump(best_t, f)
with open(OPEN_META, "wb") as f:
    pickle.dump(best_t, f)

print("\n💾 Guardado: ")
print(f" - {RECOGNIZER_PATH}")
print(f" - {ENCODER_PATH}")
print(f" - {THRESH_PATH}")
print(f" - {OPEN_META}")

Shape de X: (737, 128)
Clases: ['Santiago_Zagal', 'Vinicio_Quilumba']
➡️ Mejor configuración: {'svc__C': 10, 'svc__gamma': 'scale', 'svc__kernel': 'rbf'}

=== Reporte de clasificación ===
                  precision    recall  f1-score   support

  Santiago_Zagal       1.00      1.00      1.00        88
Vinicio_Quilumba       1.00      1.00      1.00        60

        accuracy                           1.00       148
       macro avg       1.00      1.00      1.00       148
    weighted avg       1.00      1.00      1.00       148

Matriz de confusión:
 [[88  0]
 [ 0 60]]
🔧 Umbral de confianza recomendado: 0.500

💾 Guardado: 
 - output\recognizer_svm.pickle
 - output\label_encoder.pickle
 - output\threshold.pkl
 - output\open_set_meta.pkl


In [18]:
# Inferencia en imagen con VARIOS rostros
import cv2, numpy as np, pickle
from pathlib import Path

# --- Cargar clasificador, encoder y umbral UNA SOLA VEZ ---
clf = pickle.load(open("output/recognizer_svm.pickle","rb"))
le  = pickle.load(open("output/label_encoder.pickle","rb"))

THRESH_PATH = Path("output/threshold.pkl")
if THRESH_PATH.exists():
    BEST_T = float(pickle.load(open(THRESH_PATH, "rb")))
else:
    BEST_T = 0.7  # por defecto si no hay threshold entrenado
    print(f"ℹ️ No se encontró {THRESH_PATH.name}. Usando umbral por defecto: {BEST_T}")

def _get_boxes(img, conf_det=0.5):
    """
    Devuelve una lista de cajas [ (x1,y1,x2,y2,score), ... ].
    Intenta usar detectar_caras_bgr(); si no existe, usa detectar_cara_bgr().
    """
    boxes = []
    try:
        # Si tienes una función multi-cara, úsala
        boxes = detectar_caras_bgr(img, conf_thresh=conf_det)  # debe devolver lista
        if boxes is None:
            boxes = []
    except NameError:
        # Fallback: versión una sola cara
        b = detectar_cara_bgr(img, conf_thresh=conf_det)
        if b is not None:
            boxes = [b]
    return boxes

def predecir_varios_en_imagen(path, conf_det=0.5, conf_cls=None, out_path=None):
    """
    Detecta múltiples rostros, clasifica cada uno y dibuja etiqueta/probabilidad.
    - conf_det: umbral del detector de rostros.
    - conf_cls: umbral de aceptación (si None, usa BEST_T).
    - out_path: si se pasa, guarda la imagen anotada.
    Devuelve (img_anotada, resultados) donde resultados es lista de dicts.
    """
    img = cv2.imread(path)
    if img is None:
        raise FileNotFoundError(f"No se pudo leer la imagen: {path}")

    boxes = _get_boxes(img, conf_det=conf_det)
    if not boxes:
        print("⚠️ No se detectaron rostros.")
        return img, []

    thr = float(conf_cls) if conf_cls is not None else BEST_T
    resultados = []

    for k, box in enumerate(boxes):
        (x1, y1, x2, y2, *rest) = box
        vec = embedding_cara(img, box)
        if vec is None:
            print(f"⚠️ No se pudo obtener embedding para rostro #{k+1}.")
            continue

        probs = clf.predict_proba([vec])[0]
        j = int(np.argmax(probs))
        prob = float(probs[j])
        name = le.classes_[j] if prob >= thr else "Desconocido"

        # Dibujo
        cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
        y_text = max(20, y1 - 10)
        cv2.putText(
            img, f"{name}: {prob:.2f}",
            (x1, y_text), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2
        )

        resultados.append({
            "box": (x1, y1, x2, y2),
            "score_det": rest[0] if rest else None,
            "pred_clase_idx": j,
            "pred_nombre": name,
            "pred_confianza": prob
        })

    if out_path:
        cv2.imwrite(out_path, img)
        print(f"💾 Guardado: {out_path}")

    return img, resultados


In [19]:
# Coloca una imagen 'prueba.jpg' en tu carpeta del proyecto
_ = predecir_en_imagen("imagen2.png", out_path="predicciones/prediccion.jpg")

NameError: name 'predecir_en_imagen' is not defined

In [20]:
img_annot, res = predecir_varios_en_imagen(
    "imagen2.png",
    conf_det=0.5,       # detector más/menos estricto
    conf_cls=None,      # usa el umbral aprendido (threshold.pkl)
    out_path="predicciones/imagen21.png"
)
print(res)


💾 Guardado: imagen21.png
[{'box': (321, 337, 418, 460), 'score_det': 0.9932352, 'pred_clase_idx': 5, 'pred_nombre': 'Vinicio_Quilumba', 'pred_confianza': 0.8507961302466296}]


In [26]:
img_annot, res = predecir_varios_en_imagen(
    "imagen1.jpeg",
    conf_det=0.5,       # detector más/menos estricto
    conf_cls=None,      # usa el umbral aprendido (threshold.pkl)
    out_path="predicciones/imagen12.png"
)
print(res)

💾 Guardado: imagen12.png
[{'box': (199, 309, 271, 406), 'score_det': 0.6512687, 'pred_clase_idx': 2, 'pred_nombre': 'Santiago_Zagal', 'pred_confianza': 0.8554697998297164}]
